#Analiza sentymenty z wykorzystaniem BERT i zbioru Amazon Reviews

Parametry:

In [11]:
datasetUrl = 'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Home_and_Kitchen_5.json.gz'
dataPath = './data/reviews.json.gz'

Dane karty graficznej:

In [12]:
!nvidia-smi

Thu Apr 29 17:17:02 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 461.40       Driver Version: 461.40       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 166... WDDM  | 00000000:26:00.0  On |                  N/A |
| 28%   35C    P8    21W / 120W |   1207MiB /  6144MiB |      4%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Zainstalowane biblioteki:

In [13]:
%reload_ext watermark
%watermark -v -p numpy,pandas,torch,transformers


Python implementation: CPython
Python version       : 3.8.8
IPython version      : 7.21.0

numpy       : 1.19.2
pandas      : 1.2.4
torch       : 1.8.1
transformers: 4.5.1



Importowane biblioteki:

In [14]:
import transformers
import pandas as pd
import gzip
import wget
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
import numpy as np
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

Pobieranie zbioru danych i rozpakowywanie


In [ ]:
try:
    f = open(dataPath)
except IOError:
    print("File not accessible")
    wget.download(datasetUrl, out=dataPath)
finally:
    f.close()

File not accessible


Ładowanie danych:

In [ ]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF(dataPath)

Analiza danych:


In [ ]:
df.head()

In [ ]:
df.shape


In [ ]:
df.info()


In [ ]:
sns.countplot(df.overall)
plt.xlabel('review score')